In [17]:
import pandas as pd
import numpy as np
from itertools import combinations 
import pickle as pickle
import random

In [3]:
d_all = pd.read_csv('final_proj_data_preprocessed.csv')

C:\Users\xyn94\.conda\envs\test\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2,3,4,8,11,13,15,16,17,18,19,21,22,30,38,41,42,44,47,48,50,51,74,75,87,89,94,117,119,123) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
d = d_all[['id', 'author', 'title', 'selftext']]

In [22]:
authors = d.groupby('author').id.count().to_frame()

In [23]:
same_authors = authors[authors.id>1]

In [30]:
same_authors_d = d[d.author.isin(same_authors.index)]

In [41]:
title_dict = same_authors_d.groupby('author')['title'].apply(list).to_dict()
text_dict = same_authors_d.groupby('author')['selftext'].apply(list).to_dict()

In [56]:
ds = [title_dict, text_dict]
same_author_dict = {}
for k in title_dict.keys():
    same_author_dict[k] = list(d[k] for d in ds)

In [66]:
column_names = ["seq_A", "seq_B", "label",'author']
train = pd.DataFrame(columns = column_names)
for author, t in same_author_dict.items():
    flat_list = [item for sublist in t for item in sublist]
    for c in combinations(flat_list,2):
        train = train.append({"seq_A":c[0], "seq_B":c[1], 'label':1,'author':author}, ignore_index=True)


In [68]:
with open('bert_finetuning_train_same_author.pickle', 'wb') as handle:
    pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
with open('bert_finetuning_train_same_author.pickle', 'rb') as handle:
    train_1 = pickle.load(handle)

In [50]:
train_1

,seq_A,seq_B,label,author
0,I want to run away.,I need emotional help and advice. Please consi...,1,--Lotus--
1,I want to run away.,"I know it sounds ridiculous, especially from a...",1,--Lotus--
2,I want to run away.,"This is gonna be a long one. Bear with me, and...",1,--Lotus--
3,I need emotional help and advice. Please consi...,"I know it sounds ridiculous, especially from a...",1,--Lotus--
4,I need emotional help and advice. Please consi...,"This is gonna be a long one. Bear with me, and...",1,--Lotus--
...,...,...,...,...
41198,Please help me.,I’m not completely sure if i should be posting...,1,zw3d
41199,Please help me.,So i’m currently studying for my ged test. I’m...,1,zw3d
41200,"So i’m 18, and my cousin who is a horse traine...",I’m not completely sure if i should be posting...,1,zw3d
41201,"So i’m 18, and my cousin who is a horse traine...",So i’m currently studying for my ged test. I’m...,1,zw3d


In [52]:
for row in train_1.iterrows(): 
    current_author = row[1]['author']
    d_pool = d[d.author!=current_author]
    s_title = random.choice(d_pool.title.to_list())
    s_text = random.choice(d_pool.selftext.to_list())
    r = random.choice([1,2])
    if r == 1:
        train_1 = train_1.append({"seq_A":row[1]['seq_A'], "seq_B":s_title,
                                  'label':0,'author':'paired_authors'}, ignore_index=True)
    else:
        train_1 = train_1.append({"seq_A":row[1]['seq_A'], "seq_B":s_text,
                                  'label':0,'author':'paired_authors'}, ignore_index=True)

In [53]:
with open('bert_finetuning_train_all.pickle', 'wb') as handle:
    pickle.dump(train_1, handle, protocol=pickle.HIGHEST_PROTOCOL)